In [24]:
# 原子の原子量（g/mol）
ATOMIC_WEIGHTS = {
    "H": 1.008,
    "C": 12.011,
    "N": 14.007,
    "O": 15.999,
    "F": 18.998,
    "Si": 28.085,
    "P": 30.974,
    "S": 32.06,
    "Cl": 35.45,
    # 必要に応じて追加
}

# 官能基の構成元素とその数
SUBSTITUTE_GROUPS = {
  "Boc": "C4H9OCO",
  "Me": "CH3",
  "Bn": "CH2Ph",
  "Ph": "C6H5" 
  # 必要に応じて追加
}

In [25]:
import re

def tokenize(formula):
    """
    分子式をトークンに分解します。
    トークンには元素記号、保護基、数値、括弧が含まれます。
    ex. "ClSi(CH3)3" -> ["Cl", "Si", "(", "C", "H", "3", ")", "3"]
    """
    tokens = re.findall(r"[A-Z][a-z]*|\d+|\(|\)", formula)
    return tokens

In [26]:
import re

def split_string_with_lowercase(input_string):
    # 正規表現でパターンを定義
    pattern = r'([A-Z][a-z]*)|([a-z]+)|([\(\)])|(\d+)'
    
    # マッチした部分をリストに格納
    matches = re.findall(pattern, input_string)
    
    # マッチ結果をフラットなリストに変換
    result = [item for group in matches 
                            for item in group if item]
    
    return result

In [27]:
def parse(tokens):
    """
    トークンリストをパースして、ネストされたデータ構造に変換します。
    保護基は展開されます。
    ex. ["Cl", "Si", "(", "C", "H", "3", ")", "3"]
        -> [("Cl", 1), ("Si", 1), ([("C", 1), ("H", 3)]
    """

    def parse_group(index):
        group = []
        while index < len(tokens):
            token = tokens[index]
            tmp = None
            if token == ")":
                # グループの終わりに達したら現在のグループを返す
                return group, index + 1
            if token == "(":
                # 新しいグループに到達したらグループをパースする
                tmp, index = parse_group(index + 1)
            elif token in SUBSTITUTE_GROUPS:
                # 保護基を展開
                PGtokens = tokenize(SUBSTITUTE_GROUPS[token])
                tmp = parse(PGtokens)
                index += 1
            elif token in ATOMIC_WEIGHTS:
                # 元素記号を追加
                tmp = token
                index += 1
            else:
                raise ValueError(f"Unexpected token: {token}")
            # 括弧/保護基/元素の後にくる数値を取得
            if index < len(tokens) and tokens[index].isdigit():
                count = int(tokens[index])
                index += 1
            else:
                count = 1
            group.append((tmp, count))
        return group, index

    parsed_structure, _ = parse_group(0)
    return parsed_structure

In [28]:
def calculate_molecular_weight(parsed_structure):
    """
    パースされた構造から分子量を計算します。
    ex. [("Cl", 1), ("Si", 1), ([("C", 1), ("H", 3)], 3)] -> 108.55
    """

    def recursive_calc(structure):
        total = 0
        for item in structure:
            if not isinstance(item, tuple):
                raise ValueError(f"正しい中間表現ではありません: {item}")
            key, value = item
            if isinstance(key, list):
                total += recursive_calc(key) * value
            elif isinstance(key, str):
                if key not in ATOMIC_WEIGHTS:
                    raise ValueError(f"定義されていない元素です: {key}")
                total += ATOMIC_WEIGHTS[key] * value
            else:
                raise ValueError(f"キーのタイプが不正です: {type(key)}")
        return total

    return recursive_calc(parsed_structure)

In [29]:
formula = input("Enter a chemical formula: ")
tokens = tokenize(formula)
parsed_structure = parse(tokens)
weight = calculate_molecular_weight(parsed_structure)
print(f"The molecular weight of {formula} is {weight:.2f} g/mol.")

The molecular weight of CH4 is 16.04 g/mol.
